# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [23]:
columns_list = df.columns
columns_list

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [27]:
print(dict(df.dtypes))

{'loan_amnt': dtype('float64'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'home_ownership': dtype('O'), 'annual_inc': dtype('float64'), 'verification_status': dtype('O'), 'issue_d': dtype('O'), 'loan_status': dtype('O'), 'pymnt_plan': dtype('O'), 'dti': dtype('float64'), 'delinq_2yrs': dtype('float64'), 'inq_last_6mths': dtype('float64'), 'open_acc': dtype('float64'), 'pub_rec': dtype('float64'), 'revol_bal': dtype('float64'), 'total_acc': dtype('float64'), 'initial_list_status': dtype('O'), 'out_prncp': dtype('float64'), 'out_prncp_inv': dtype('float64'), 'total_pymnt': dtype('float64'), 'total_pymnt_inv': dtype('float64'), 'total_rec_prncp': dtype('float64'), 'total_rec_int': dtype('float64'), 'total_rec_late_fee': dtype('float64'), 'recoveries': dtype('float64'), 'collection_recovery_fee': dtype('float64'), 'last_pymnt_amnt': dtype('float64'), 'next_pymnt_d': dtype('O'), 'collections_12_mths_ex_med': dtype('float64'), 'policy_code': dtype('float64'), 'applicatio

In [29]:
# I'm getting an error because some of these features are still objects.
# I want a list of just the columns that are objects:
df.select_dtypes('object').columns

Index(['home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
       'hardship_flag', 'debt_settlement_flag'],
      dtype='object')

In [79]:
# Use one hot encoding to transform these into integers
df_encoded = pd.get_dummies(df, columns=['home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
       'hardship_flag', 'debt_settlement_flag'])

df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [80]:
# Checking that all the object data types (except loan status) are gone
df_encoded.select_dtypes('object').columns

Index([], dtype='object')

In [84]:
# I'm looking for the new name of the column for loan status
print(dict(df_encoded.dtypes))

{'loan_amnt': dtype('float64'), 'int_rate': dtype('float64'), 'installment': dtype('float64'), 'annual_inc': dtype('float64'), 'dti': dtype('float64'), 'delinq_2yrs': dtype('float64'), 'inq_last_6mths': dtype('float64'), 'open_acc': dtype('float64'), 'pub_rec': dtype('float64'), 'revol_bal': dtype('float64'), 'total_acc': dtype('float64'), 'out_prncp': dtype('float64'), 'out_prncp_inv': dtype('float64'), 'total_pymnt': dtype('float64'), 'total_pymnt_inv': dtype('float64'), 'total_rec_prncp': dtype('float64'), 'total_rec_int': dtype('float64'), 'total_rec_late_fee': dtype('float64'), 'recoveries': dtype('float64'), 'collection_recovery_fee': dtype('float64'), 'last_pymnt_amnt': dtype('float64'), 'collections_12_mths_ex_med': dtype('float64'), 'policy_code': dtype('float64'), 'acc_now_delinq': dtype('float64'), 'tot_coll_amt': dtype('float64'), 'tot_cur_bal': dtype('float64'), 'open_acc_6m': dtype('float64'), 'open_act_il': dtype('float64'), 'open_il_12m': dtype('float64'), 'open_il_24m'

## Split the Data into Training and Testing

In [100]:
# Create our features
X = df_encoded.drop(columns=["loan_status_low_risk", "loan_status_high_risk"])

# Create our target
y = df_encoded[["loan_status_low_risk"]]

In [101]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [102]:
# Check the balance of our target values
y['loan_status_low_risk'].value_counts()

1    68470
0      347
Name: loan_status_low_risk, dtype: int64

In [103]:
y = y['loan_status_low_risk']

In [104]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

# I'm going to add the stratify argument to ensure the model has the same proportion of x and y
# values in the train and test sets as in the original y array.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [105]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [106]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [107]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [108]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=500, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [109]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

acc_score = balanced_accuracy_score(y_test, predictions)
random_forest_acc = f"Random Forest Balanced Accuracy Score : {acc_score}"
print(random_forest_acc)

Random Forest Balanced Accuracy Score : 0.6605982410126868


In [110]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Calculating the confusion matrix first
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Then display the confusion matrix
print("Random Forest Confusion Matrix")
display(cm_df)

Random Forest Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,28,59
Actual 1,11,17107


In [111]:
# Print the imbalanced classification report
print("Random Forest Classification Report")
print(classification_report(y_test, predictions))

Random Forest Classification Report
              precision    recall  f1-score   support

           0       0.72      0.32      0.44        87
           1       1.00      1.00      1.00     17118

    accuracy                           1.00     17205
   macro avg       0.86      0.66      0.72     17205
weighted avg       1.00      1.00      1.00     17205



In [112]:
# Printing all reports together
print(random_forest_acc)

print("Random Forest Confusion Matrix")
display(cm_df)

print("Classification Report")
print(classification_report(y_test, predictions))

Random Forest Balanced Accuracy Score : 0.6605982410126868
Random Forest Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,28,59
Actual 1,11,17107


Classification Report
              precision    recall  f1-score   support

           0       0.72      0.32      0.44        87
           1       1.00      1.00      1.00     17118

    accuracy                           1.00     17205
   macro avg       0.86      0.66      0.72     17205
weighted avg       1.00      1.00      1.00     17205



In [113]:
# List the features sorted in descending order by feature importance

# Get the feature array first
importances = rf_model.feature_importances_

# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.07811979300501612, 'total_rec_prncp'),
 (0.07596140310783511, 'total_rec_int'),
 (0.06641690840377341, 'total_pymnt_inv'),
 (0.06398647002698087, 'total_pymnt'),
 (0.06008668281793827, 'last_pymnt_amnt'),
 (0.021582719710088627, 'out_prncp_inv'),
 (0.020838390382238354, 'out_prncp'),
 (0.020227467330126467, 'installment'),
 (0.01671718490945773, 'dti'),
 (0.01642847701422232, 'mo_sin_old_rev_tl_op')]

### Easy Ensemble Classifier

In [114]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

ensemble_model = EasyEnsembleClassifier(n_estimators=500, random_state=1)
ensemble_model = ensemble_model.fit(X_train_scaled, y_train)
predictions = ensemble_model.predict(X_test_scaled)

In [115]:
# Calculated the balanced accuracy score
ensemble_acc_score = balanced_accuracy_score(y_test, predictions)
ensemble_acc = f"Easy Ensemble Balanced Accuracy Score : {acc_score}"
print(ensemble_acc)

Easy Ensemble Balanced Accuracy Score : 0.6605982410126868


In [116]:
# Display the confusion matrix
# Calculating the confusion matrix first
cm = confusion_matrix(y_test, predictions)
ensemble_cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Then display the confusion matrix
print("Easy Ensemble Confusion Matrix")
display(ensemble_cm_df)

Easy Ensemble Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,79,8
Actual 1,947,16171


In [117]:
# Print the imbalanced classification report
print("Easy Ensemble Classification Report")
print(classification_report(y_test, predictions))

Easy Ensemble Classification Report
              precision    recall  f1-score   support

           0       0.08      0.91      0.14        87
           1       1.00      0.94      0.97     17118

    accuracy                           0.94     17205
   macro avg       0.54      0.93      0.56     17205
weighted avg       0.99      0.94      0.97     17205



In [77]:
print(ensemble_acc)
print(random_forest_acc)

Easy Ensemble Balanced Accuracy Score : 0.6605982410126868
Random Forest Balanced Accuracy Score : 0.6605982410126868


### Final Questions

1. Which model had the best balanced accuracy score?

    Both the Random Forest and the Easy Ensemble models have the exact same accuracy score, 0.06605982...
    I tweaked the models and ran them twice, and both times the accuracy scroe was the same.

2. Which model had the best recall score?

    Easy Ensemble had the best macro average, or unweighted, recall score by far: a macro average of 
    0.93, as compared to the Random Forest Model's macro average recall score of 0.66.

    However, Random Forest had the best weighted average recall score at 1.0 to Easy Ensemble's 0.94 weighted average recall score.

    I think the weighted average recall is more convincing because it takes into account the support.

3. Which model had the best geometric mean score?

    The geometric mean score is listed as F-1, and represents the "Harmonic mean of precision and recall for a more balanced summarization of model performance." (See https://towardsdatascience.com/micro-macro-weighted-averages-of-f1-score-clearly-explained-b603420b292f). 
    
    The micro average is the best metric for this, which is why it's listed just as the "accuracy" metric. Random Forest had an F-1 accuracy score of 1.0, which was higher than Easy Ensemble's F-1 accuracy score of 0.94.

4. What are the top three features?

    1. Total Received Principle ('total_rec_prncp')
    2. Total Received Principle ('total_rec_int')
    3. Total Payment Invoice ('total_pymnt_inv')